<a href="https://colab.research.google.com/github/kakopo/NairobiKenyaChapter_InvisibleDisabilitiesCommunicator/blob/main/RAG_NKC_IDC_LC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Utilizing Retrieval Augmented Generation for Nairobi Invisible Disabilities Classifier.**


In [ ]:
!pip install langchain
!pip install --upgrade --quiet "unstructured[all-docs]"
!pip install -U langchain-community
!pip install -qq langchain langchain-nvidia-ai-endpoints gradio
!pip install faiss-cpu
!pip install langchain_groq
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 17.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.6/514.6 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from pathlib import Path
# from langchain_openai import ChatOpenAI,OpenAIEmbeddings
import os
from dotenv import load_dotenv

import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [ ]:
from langchain_groq import ChatGroq # Import GroqChat instead of ChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings # Import HuggingFaceEmbeddings

In [ ]:
from google.colab import userdata
HF_TOKEN=userdata.get('HF_TOKEN')
GROQ_API_KEY=userdata.get('GROQ_API_KEY')

In [ ]:
# Load environment variables from a .env file
load_dotenv()

# Set the API key environment variables

os.environ["HUGGINGFACEHUB_API_TOKEN"] = HF_TOKEN

os.environ["GROQ_API_KEY"] = GROQ_API_KEY

llm = ChatGroq(model="mixtral-8x7b-32768")

In [ ]:
import pandas as pd
from langchain_community.document_loaders.csv_loader import CSVLoader

# URL of the CSV file on GitHub
url = "https://raw.githubusercontent.com/OmdenaAI/NairobiKenyaChapter_InvisibleDisabilitiesCommunicator/main/Data/Matatu_Routes.csv"

# Read the CSV file into a pandas DataFrame
data = pd.read_csv(url)

data.head(5)

,Matatu Route Number,Destination/ Bus-Stops,Bus pickup point in Nairobi
0,Matatu Route Number 1,"Ngong Road, Citam/NPC Valley road, The Junctio...",General Post Office/ GPO
1,Matatu Route Number 2,"Ngong Road, Citam/NPC Valley Rd, The junction ...",GPO or KBS stage
2,Matatu Route Number 3,"Ngong Road, Adam;s Arcade, Dagoretti corner",GPO
3,Matatu Route Number 6,Eastleigh Area/ Pangani Girls/ Pangani Shoppin...,Board the Matatu from Accra Rd or Along Tom M...
4,Matatu Route Number 7C,"Community, Upper hill Area, Kenyatta National ...",Board the Citi Hoppa or KBS buses at Kencom Ho...


In [ ]:
# Save the DataFrame to a temporary CSV file
data.to_csv('temp_matatu_routes.csv', index=False)

In [ ]:
# Now, load the data from the temporary file using CSVLoader
loader = CSVLoader(file_path='temp_matatu_routes.csv')  # Use the temporary file path
docs = loader.load_and_split()

In [ ]:
embeddings = HuggingFaceEmbeddings()
index = faiss.IndexFlatL2(len(HuggingFaceEmbeddings().embed_query(" ")))
vector_store = FAISS(
    embedding_function=HuggingFaceEmbeddings(),
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

<ipython-input-16-7d2217492fc2>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
<ipython-input-16-7d2217492fc2>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-16-7d2217492fc2>:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  index = faiss.IndexFlatL2(len(HuggingFaceEmbeddings().embed_query(" ")))
<ipython-input-16-7d2217492fc2>:4: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding_function=HuggingFaceEmbeddings(),


In [ ]:
vector_store.add_documents(documents=docs)

['b7f34924-ab24-454e-b770-12b60b9b6855',
 'f52dff12-0b5e-4009-9186-8eaace9585df',
 '2aaeca2b-4578-43bf-869d-25c8f1b3c312',
 'f3001ac7-1c69-4706-bfee-7a74aed9777e',
 'fa36001b-2146-4d96-958e-ccc2df02b6c0',
 'd5475328-f4e6-46b7-b89b-8dcd293b86cb',
 'c114bbac-d047-4c3d-8df0-c3c74014bc40',
 'ef58f5d3-6749-4ae6-95fd-5d351968c297',
 'd8e6b468-b220-484f-905c-b00eb823f6c0',
 '831e6352-2684-49c5-93f2-48957fdeb104',
 'd4753ef3-de56-4e5f-ad26-7a643c716052',
 '9d93a409-74e4-40be-947b-98f6a76e0ffc',
 'eff9726d-bd64-435a-9d5a-0c170caa58af',
 '819ce31a-f72f-4ed8-84b4-b3ce5553933d',
 '9a7ee24f-6635-451f-9c2a-dda779a624b6',
 '65408896-32e3-41cf-98d7-716ac27e1957',
 '63e963e4-3e05-4a39-bb99-a18cd9cd6857',
 '01d54851-1033-4fb7-b008-1241872d256e',
 'c4637b08-2b31-41a9-83cf-bfee1358a205',
 '4abe4fab-7b45-4c61-b868-1104d93c978a',
 '30797d31-3722-487a-9018-d2d28d649475',
 '31b66b6d-0ff1-4dac-a3c0-50c37160a780',
 '0c070bd5-3fcf-489d-8335-234f7ebd31b1',
 'ee9ce07d-3d2b-4f89-9592-85ca88b5ce48',
 'f6e84f3f-a55c-

In [ ]:
retriever = vector_store.as_retriever()

# Set up system prompt
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),

])

# Create the question-answer chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
answer= rag_chain.invoke({"input": "what is Matatu Route Number 1?"})
answer['answer']

'Matatu Route Number 1 starts from the General Post Office (GPO) in Nairobi and has destinations that include Ngong Road, Citam/NPC Valley Road, The Junction Mall, and Dagoretti.'

In [ ]:
answer= rag_chain.invoke({"input": "what Matatu Route Number should I take if my destination is Eastleigh Area?"})
answer['answer']

'You should take Matatu Route Number 9 if your destination is the Eastleigh Area. You can board the matatu at Tom Mboya Street or opposite Nairobi County Fire Station.'

In [ ]:
answer= rag_chain.invoke({"input": "what Matatu should I take if I want to go to Ongata Rongai?"})
answer['answer']

'To go to Ongata Rongai, you should take Matatu Route Number 125/126. You can board it from Railways bus station.'

In [ ]:
answer= rag_chain.invoke({"input": "what is Matatu Route Number 23?"})
answer['answer']

'Matatu Route Number 23 starts from Kangemi, goes through Westlands, Uhuru Highway, Kangemi, ABC place, Safaricom House, and returns to Uhuru highway. You can board this matatu at Odeon cinema / Latema Road off tom Mboya street in Nairobi.'